In [ ]:
# CHAINS! It usually combines LLM with prompts, and with this you can really use this to generate a great SEQUENCE OF EVENTS TOGETHER!
import os
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read the local .env file
openai.api_key = os.environ['OPENAI_API_KEY'] # this is now the API key to use

testEnvString = os.environ['TEST_ENV'] # test to see if .env is working
print(testEnvString)

In [ ]:
# You can read several different kinds of inputs here!
# For example, let's load in a pandas dataframe
import pandas as pd
df = pd.read_csv('./assets/TopicsList.csv')

df.head()

# Let's input a few things
# model
from langchain_openai.chat_models import ChatOpenAI
# prompts
from langchain.prompts import ChatPromptTemplate
# chains
from langchain.chains import LLMChain
model = 'gpt-4o'

llm = ChatOpenAI(temperature=0.1, model=model) # initialize our LLM

prompt = ChatPromptTemplate.from_template(
    "What is the best topic to study, if I want to brush up on {studyTopic}?"
)

chain = LLMChain(llm=llm, prompt=prompt)


In [ ]:
studyTopic = "string manipulation and concatenation"
print(chain.run(studyTopic))

# LLM chain is quite useful

In [ ]:
# SEQUENTIAL CHAINS
# Chains that run one after each other. The idea is to combine multiple chains where the output of one chain, is the input of the next

# TWO TYPES:
# 1. SimpleSequentialChain: single input/output
# 2. SequentialChain: multiple inputs/outputs

In [ ]:
from langchain.chains import SimpleSequentialChain

llm = ChatOpenAI(temperature=0.8, model=model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [ ]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True)

In [ ]:
# Run these in conjunction with another to make one input another, make one output inform another input, etc. etc.
product = "soap that smells like Aloe?"
overall_simple_chain.run(product)

In [ ]:
# MULTIPLE INPUTS? MULTIPLE OUTPUTS?
# we can use the regular SequentialChain for this part
from langchain.chains import SequentialChain


In [ ]:
# Essentially, each LLMChain is like a "node" along the chain, and can produce an output_key that must be EXACTLY THAT in the next chain node
# The essential parts for each LLMChain are:

# 1. need llm=llm, to note which model to use
# 2. need prompt=prompt, can be created from template
# 3. recommend output_key, so that the input can be used in a subsequent chain

# SEVERAL DIFFERENT INPUTS can be used to create a bunch of these LLM calls! :))

In [ ]:
# ROUTER CHAINS <- this might be super relevant to you

# this is a way to route and find the best chain, based off of the input
# The router chain can choose the best CHAIN to go down!

# for example, each kind of chain can be optimized based on different topics
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [ ]:
# specific type of chain to navigate multiple prompt templates
from langchain.chains.router import MultiPromptChain

# this is uses the LLM itself to choose the right chain. the output parser also 
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
llm = ChatOpenAI(temperature=0, model=model)

# prompt_info is used to choose the different potential chains to use

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    # you essentially create a chain element, to start each branching path
    chain = LLMChain(llm=llm, prompt=prompt)

    # then, you add it to the list/hashmap of potential destination chains the router can go anywhere
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [ ]:
chain.run("What is the best data structure if I want to implement a way to always have the smallest value handy and accessible?")

In [ ]:
print(chain.run("What is the pythagorean theorem?"))